Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [676]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import numpy as np


## The *Nim* and *Nimply* classes

In [677]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [678]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [679]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [680]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [681]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


In [682]:
# values is a vector with the probabilities to choose each rule


def first_strategy(state, values):
    # find all the possible secure moves (as the optimal strategy, but we don't choose the move randomly)
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    rule = np.random.choice(len(values), p=values)
    if rule == 0:  # choose the secure moves which take from the last possible row
        return max(spicy_moves, key=lambda e: e.row)
    if rule == 1:  # choose the secure moves which take more objects
        return max(spicy_moves, key=lambda e: e.num_objects)
    if rule == 2:  # choose the secure moves which take less objects
        return min(spicy_moves, key=lambda e: e.num_objects)
    return min(
        spicy_moves, key=lambda e: e.row
    )  # choose the secure moves which take from the first possible row


def second_strategy(state: Nim, values):
    rule = np.random.choice(len(values), p=values)

    # with N_rule=5: debug, it converges to the optimal solution
    # if rule==4:
    #     return optimal(state)

    possible_moves = [  # find all the possible moves
        [i, state._rows[i] if state._k is None else min(state._rows[i], state._k)]
        for i in range(len(state._rows))
        if state._rows[i] != 0
    ]
    if rule == 0:  # choose moves which take from the last possible row
        ply = max(possible_moves, key=lambda e: e[0])
    elif rule == 1:  # choose moves which take more objects
        ply = max(possible_moves, key=lambda e: e[1])
    elif rule == 2:  # choose moves which take less objects
        ply = min(possible_moves, key=lambda e: e[1])
    elif rule ==3:  # choose moves which take from the first possible row
        ply = min(possible_moves, key=lambda e: e[0])

    return Nimply(ply[0], ply[1])

In [683]:
μ = 10
λ = 20
σ = 1
N_rule = 4
N_Train = 100

# the fitness is given from the number of match win against the optimal strategy with the probability to choose one of the four possible rule in values vector

def fitness(values):
    count = 0

    for _ in range(N_Train):
        nim = Nim(5)
        player = 0
        while nim:
            ply = optimal(nim) if player == 0 else second_strategy(nim, values)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            count += 1

    return (count*100)/N_Train

# mutation more similar to the pseudocode in "essential"
# def mutation(population):
#     offspring = []
#     for p in population:
#         for _ in range(λ//μ):
#             off = (p[0], np.abs(np.random.normal(loc=p[-1], scale=0.2)))
#             offspring_noise = np.abs(
#                 np.random.normal(loc=off[0], scale=off[1])
#             )  # find the gaussian noise
#             offspring_noise = list(
#                 offspring_noise / np.sum(offspring_noise)
#             )  # the sum of the new probabilities must be 1
#             offspring.append((offspring_noise, fitness(offspring_noise), off[1]))

#     return offspring

def mutation(population):
    offspring = []

    for _ in range(λ):
        p = population[np.random.randint(0, μ)]
        off =(p[0], np.abs(np.random.normal(loc=p[-1], scale=0.2)))
        offspring_noise = np.abs(
            np.random.normal(loc=off[0], scale=off[1])
        )  # find the gaussian noise
        offspring_noise = list(
            offspring_noise / np.sum(offspring_noise)
        )  # the sum of the new probabilities must be 1
        offspring.append((offspring_noise, fitness(offspring_noise),off[1]))

    return offspring

In [684]:
def ES_algorithm():
    population = []
    best_fitness = None
    best_probabilities = None

    for _ in range(μ):
        probabilities = list(np.random.dirichlet(np.ones(N_rule)))
        population.append((probabilities, fitness(probabilities), σ))
    step = 0

    while best_fitness is None or (best_fitness < 50.0 and step < 100):
        offspring = mutation(population)

        # parents compete with the new elements
        offspring.extend(population)
        tmp_pop = sorted(offspring, key=lambda e: e[1])

        if best_fitness is None or tmp_pop[-1][1] > best_fitness:
            best_fitness = tmp_pop[-1][1]
            best_probabilities = tmp_pop[-1][0]

        # choose the μ best elements (considering the results of the fitness) to add them to the population
        population = tmp_pop[-μ:]

        step += 1

    return best_probabilities, best_fitness


adaptive_player, rating = ES_algorithm()

print(f"Best Probabilities: {adaptive_player}\nWin percentage:{rating}%")


def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    return second_strategy(state, adaptive_player)

Best Probabilities: [0.13863852464267112, 0.8319575645291863, 0.02287571494440402, 0.006528195883738516]
Win percentage:53.0%


## Oversimplified match

In [685]:
logging.getLogger().setLevel(logging.INFO)

strategy = (optimal, adaptive)

nim = Nim(5)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=8)
INFO:root:status: <1 3 5 7 1>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=7)
INFO:root:status: <1 3 5 0 1>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=2)
INFO:root:status: <1 1 5 0 1>
INFO:root:ply: player 1 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 1 5 0 1>
INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <0 1 5 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=5)
INFO:root:status: <0 1 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 0 0 0 0>
INFO:root:status: Player 1 won!
